# Infering Experiments

## Testing pickled objects

In [24]:
import os
import pickle
import numpy as np
import pandas as pd

In [25]:
with open('PPSNResults/FIBERS2/Goal1/standard_no_noise/models/1', 'rb+') as file:
    model = pickle.load(file)
model

FIBERS(covariates=[], diversity_pressure=0, iterations=100, max_bin_size=100,
       max_mutation_prob=0.1, merge_prob=0.0, pop_clean='group_strata',
       random_seed=1)

In [26]:
with open('PPSNResults/FIBERS1/Goal1/standard_no_noise/models/1', 'rb+') as file:
    model = pickle.load(file)
model

FIBERS(covariates=[], elitism_parameter=0.1, evolving_probability=0.0,
       iterations=100, label_name='Censoring',
       max_number_of_groups_with_feature=10, max_threshold=4,
       min_features_per_group=1, mutation_probability=0.1, random_seed=1)

## Making Important Tables

In [27]:
from glob import glob

In [28]:
list_of_exp_folders = glob("PPSNResults/*/*/*/models", recursive = True)
list_of_exp_folders

['PPSNResults/FIBERS1/Goal1/standard_with_noise/models',
 'PPSNResults/FIBERS1/Goal1/standard_no_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_0_with_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_1_no_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_1_with_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_0_no_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_2_no_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_2_with_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_4_with_noise/models',
 'PPSNResults/FIBERS1/Goal4/threshold_4_no_noise/models',
 'PPSNResults/FIBERS2/Goal6/covariates_dataset_3/models',
 'PPSNResults/FIBERS2/Goal6/covariates_dataset_2/models',
 'PPSNResults/FIBERS2/Goal6/covariates_dataset_1/models',
 'PPSNResults/FIBERS2/Goal1/standard_with_noise/models',
 'PPSNResults/FIBERS2/Goal1/standard_no_noise/models',
 'PPSNResults/FIBERS2/Goal3/standard_with_noise/models',
 'PPSNResults/FIBERS2/Goal3/standard_no_noise/models',
 'PPSNResults/FIBE

In [29]:
path = 'PPSNResults/FIBERS1/Goal1/standard_with_noise/models/model_table_1.csv'

In [30]:
tdf = pd.read_csv(path)
columns = list(tdf.T.iloc[0])

In [31]:
def data_stats(data, confidence=0.95):

  mean = data.mean()
  std = data.std()

  t_value = np.abs(np.percentile(np.random.standard_t(len(data) - 1), (1 - confidence) / 2))

  margin_of_error = t_value * std / np.sqrt(len(data))

  lower_bound = mean - margin_of_error
  upper_bound = mean + margin_of_error

  return mean, std, lower_bound, upper_bound

In [32]:
def get_summary_sats(experiment_df):
    experiment_df[['HR CI 5%', 'HR CI 95%']] = experiment_df['HR CI'].str.split('-', expand=True)
    ex_df = experiment_df.drop(['Features in Bin', 'HR CI', 'Dataset Filename'], axis=1)
    ex_df = ex_df.apply(pd.to_numeric, errors='raise')
    mean, stdev, lower_bound, upper_bound = data_stats(ex_df)
    return mean, stdev, lower_bound, upper_bound

In [33]:
files_list = list()
mean_list, stdev_list, lower_bound_list, upper_bound_list = list(), list(), list(), list()
exp_list = list()
for dir in list_of_exp_folders:
    files = list()
    experiment_df = pd.DataFrame()
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            path = os.path.join(dir, file)
            files.append(path)
            tdf = pd.read_csv(path)
            experiment_df = pd.concat([experiment_df, pd.DataFrame([list(tdf.T.iloc[1])])], 
                                      ignore_index=True)
    experiment_df.columns=columns
    experiment_df.to_csv('/'.join(dir.split('/')[:-1]) + '/experiment_table.csv')
    print('/'.join(dir.split('/')[:-1]) + '/experiment_table.csv')
    files_list.append(files)
    mean, stdev, lower_bound, upper_bound = get_summary_sats(experiment_df)
    mean_list.append(mean)
    stdev_list.append(stdev)
    lower_bound_list.append(lower_bound)
    upper_bound_list.append(upper_bound)
    exp_list.append(dir)
len(files_list)

PPSNResults/FIBERS1/Goal1/standard_with_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal1/standard_no_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_0_with_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_1_no_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_1_with_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_0_no_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_2_no_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_2_with_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_4_with_noise/experiment_table.csv
PPSNResults/FIBERS1/Goal4/threshold_4_no_noise/experiment_table.csv
PPSNResults/FIBERS2/Goal6/covariates_dataset_3/experiment_table.csv
PPSNResults/FIBERS2/Goal6/covariates_dataset_2/experiment_table.csv
PPSNResults/FIBERS2/Goal6/covariates_dataset_1/experiment_table.csv
PPSNResults/FIBERS2/Goal1/standard_with_noise/experiment_table.csv
PPSNResults/FIBERS2/Goal1/standard_no_noise/e

29

In [34]:
exp_df = pd.DataFrame(exp_list)
exp_df = exp_df[0].str.split('/', expand=True)
exp_df.columns = ['OutputFolder', 'Version', 'Goal', 'Noise', 'Models']
exp_df = exp_df[['Goal', 'Version', 'Noise']]
exp_df['Dataset'] = exp_df['Noise'] + '.csv'
exp_df['Noise'] = exp_df['Noise'].apply(lambda x: 'Yes' if 'with' in x else 'No')
exp_df

,Goal,Version,Noise,Dataset
0,Goal1,FIBERS1,Yes,standard_with_noise.csv
1,Goal1,FIBERS1,No,standard_no_noise.csv
2,Goal4,FIBERS1,Yes,threshold_0_with_noise.csv
3,Goal4,FIBERS1,No,threshold_1_no_noise.csv
4,Goal4,FIBERS1,Yes,threshold_1_with_noise.csv
5,Goal4,FIBERS1,No,threshold_0_no_noise.csv
6,Goal4,FIBERS1,No,threshold_2_no_noise.csv
7,Goal4,FIBERS1,Yes,threshold_2_with_noise.csv
8,Goal4,FIBERS1,Yes,threshold_4_with_noise.csv
9,Goal4,FIBERS1,No,threshold_4_no_noise.csv


In [35]:
mean_df = pd.concat([exp_df, pd.DataFrame(mean_list)], axis=1)
stdev_df = pd.concat([exp_df, pd.DataFrame(stdev_list)], axis=1)
lower_bound_df = pd.concat([exp_df, pd.DataFrame(lower_bound_list)], axis=1)
upper_bound_df = pd.concat([exp_df, pd.DataFrame(upper_bound_list)], axis=1)

In [36]:
mean_df = mean_df.sort_values(['Goal', 'Version', 'Noise'])
stdev_df = stdev_df.sort_values(['Goal', 'Version', 'Noise'])
lower_bound_df = lower_bound_df.sort_values(['Goal', 'Version', 'Noise'])
upper_bound_df = upper_bound_df.sort_values(['Goal', 'Version', 'Noise'])
mean_df

,Goal,Version,Noise,Dataset,Number of P,Number of R,Bin Size,Pred Ratio,Birth Iteration,Iterations to Ideal Solution,...,Count At/Below Threshold,Count Above Threshold,Group Ratio,Log-Rank p-value,Threshold,Accuracy,Residual,Residual p-value,HR CI 5%,HR CI 95%
1,Goal1,FIBERS1,No,standard_no_noise.csv,4.85,0.00,4.85,1.000000,NaN,NaN,...,5205.95,4794.05,0.520595,0.000000e+00,0.00,0.979405,NaN,NaN,9.506074,10.702985
0,Goal1,FIBERS1,Yes,standard_with_noise.csv,4.60,0.00,4.60,1.000000,NaN,NaN,...,5273.10,4726.90,0.527310,1.403332e-301,0.00,0.972690,NaN,NaN,2.254005,2.466176
14,Goal1,FIBERS2,No,standard_no_noise.csv,10.00,0.00,10.00,1.000000,22.20,22.200000,...,5000.00,5000.00,0.500000,0.000000e+00,0.00,1.000000,NaN,NaN,10.546872,11.897023
13,Goal1,FIBERS2,Yes,standard_with_noise.csv,7.00,0.00,7.00,1.000000,23.35,NaN,...,4928.00,5072.00,0.492800,0.000000e+00,0.00,0.992800,NaN,NaN,2.307335,2.524690
28,Goal2,FIBERS2,No,standard_no_noise.csv,10.00,0.00,10.00,1.000000,22.45,22.450000,...,5000.00,5000.00,0.500000,0.000000e+00,0.00,1.000000,NaN,NaN,10.546872,11.897023
27,Goal2,FIBERS2,Yes,standard_with_noise.csv,7.00,0.00,7.00,1.000000,20.00,NaN,...,4928.00,5072.00,0.492800,0.000000e+00,0.00,0.992800,NaN,NaN,2.307335,2.524690
16,Goal3,FIBERS2,No,standard_no_noise.csv,10.00,0.00,10.00,1.000000,34.75,34.750000,...,5000.00,5000.00,0.500000,0.000000e+00,0.00,1.000000,NaN,NaN,10.546872,11.897023
15,Goal3,FIBERS2,Yes,standard_with_noise.csv,7.00,0.00,7.00,1.000000,29.50,NaN,...,4928.00,5072.00,0.492800,0.000000e+00,0.00,0.992800,NaN,NaN,2.307335,2.524690
3,Goal4,FIBERS1,No,threshold_1_no_noise.csv,5.40,0.30,5.70,0.972222,NaN,NaN,...,5795.30,4204.70,0.579530,0.000000e+00,1.15,0.918490,NaN,NaN,2.652155,2.921898
5,Goal4,FIBERS1,No,threshold_0_no_noise.csv,5.00,0.00,5.00,1.000000,NaN,NaN,...,5191.00,4809.00,0.519100,0.000000e+00,0.00,0.980900,NaN,NaN,9.568856,10.775027


In [37]:
output_folder = 'PPSNResults/'
mean_df.to_csv(output_folder + 'MasterTable_Mean.csv')
stdev_df.to_csv(output_folder + 'MasterTable_STD.csv')
lower_bound_df.to_csv(output_folder + 'MasterTable_CI_Lower.csv')
upper_bound_df.to_csv(output_folder + 'MasterTable_CI_Upper.csv')